# Assess potential urban heatwave risks

This guide is separated into two different thematical sections. The first section describes how a heatwave risk map could be derived, while the second how to integrade land use information into the latter.

## Section A: Heatwave risk map

To produce the heatwave risk map, we will follow the workflow proposed by the EU-funded project [CLIMAAX](https://handbook.climaax.eu/notebooks/workflows/HEATWAVES/01_Urban_heatwaves/heatwave_intro.html), with some modifications in the libraries and code flow used.

To implement this workflow, we will need **Landsat 8 surface temperature raster maps** and **population density maps** disaggregated by sex and age.

### Population density maps

Raster maps representing age and sex structures can be obtained from [WorldPop Hub](https://hub.worldpop.org/geodata/listing?id=138). These maps have a **100 m spatial resolution** and are available for the years **2015 to 2030**. It is recommended to download a map corresponding to a year is which a **national population survey**, so that the validity of the raster data can be assessed.

A .zip file will be downloaded. When opended, it contains raster maps for **males (m), females (f)** and the **total population (t)**, each divided into the age groups: 00, 01, 05, 10, and so on, up to 90, in five years intervals.

### Section A - Step1: Obtain the Landsat 8 images

Landsat 8 raster maps can be obtained from [RSLab](https://rslab.gr/Landsat_LST.html). From the *Select Landast* drop down menu, choose **Landsat 8** and then from the *Select Emissivity* menu, select **MODIS**. Use the **Draw Polygon** tool to delineate your study area. Alternatively, you can upload a .KML file, using the *Load KML* option; however, this feature may not always work properly, so the first option is recommended.

Once you have drawn your study area (or successfully upload your .KML file), select the period during which your area is more likelly to experience heatwaves, and click on the *Calculate LST* button.

For each resulting product, click *Show* to verify that your study area is fully covered by it. If it is not, note the image name. After completing the check, click *All Images* button to download all the products.

The images will be compressed and downloaded as a .zip file named *AllImages_LST.zip*. Open the folder and delete any images that do not fully cover your study area.

For more information refer to the *Manuals & Updates* section available on [RSLab](https://rslab.gr/Landsat_LST.html).


### Section A - Step2: Landsat 8 images preprocess

Initially, create all the neccessary paths to read and save the derived data.


In [ ]:
from pathlib import Path

# Define main and subdirectories
main_path = Path("main_path_to_your_project_directory")
lst_images = main_path / "lst_images"
study_area = main_path / "study_area"
output_folder = main_path / "output_folder"

# Create directories if they don't exist
for folder in [main_path, lst_images, study_area, output_folder]:
    folder.mkdir(parents=True, exist_ok=True)

print("All required directories are ready.")

### Section A - Step3: Load and clip the products to the study area boundaries

**Before proceeding, make sure that the file *AllImages_LST.zip* file has been transferred to the *lst_images* directory you created earlier.**

In [ ]:
import rioxarray as rxr
import geopandas as gpd
from io import BytesIO
from shapely.ops import unary_union
import zipfile

# Read vector layer once (in its native CRS)
area_vector = 'area_vector.shp' # Can be gpk, shp
gdf_orig = gpd.read_file(f"{study_area}/{area_vector}")

# Optional: dissolve multiple polygons into a single boundary
# (This avoids tiny slivers / gaps when clipping many rasters)
if len(gdf_orig) > 1:
    dissolved_geom = unary_union(gdf_orig.geometry)
    gdf_orig = gpd.GeoDataFrame(geometry=[dissolved_geom], crs=gdf_orig.crs)

# Fix potential geometry issues (self-intersections)
gdf_orig["geometry"] = gdf_orig.buffer(0)

# Loop ZIPs
for zip_file in sorted(lst_images.glob("*.zip")):
    print(f"\nProcessing ZIP: {zip_file.name}\n")
    with zipfile.ZipFile(zip_file, "r") as archive:

    # List .tif entries
        tif_names = [n for n in archive.namelist()]
        if not tif_names:
            print("\nNo .tif files found.\n")
            continue

        for tif_name in tif_names:
            print(f"\nReading & clipping: {tif_name}\n")

            # Read raster in-memory
            with archive.open(tif_name) as f:
                data = f.read()
            raster = rxr.open_rasterio(BytesIO(data), masked=True)

            # Skip rasters without CRS
            if raster.rio.crs is None:
                print("\nWARNING: Raster has no CRS. Skipping.\n")
                continue

            # Reproject vector to the raster CRS
            gdf = gdf_orig.to_crs(raster.rio.crs)

            # Clip (drop=True trims the array to the geometry; invert=False keeps inside)
            try:
                clipped = raster.rio.clip(
                    gdf.geometry, gdf.crs, drop=True, invert=False
                )
            except Exception as e:
                print(f"\nERROR clipping {tif_name}: {e}\n")
                continue

            # Optional: carry a meaningful name as a coordinate
            # (helps later when stacking/identifying)
            clipped = clipped.assign_coords(
                source_zip=zip_file.name,
                source_path_in_zip=tif_name
            )

            # Save clipped raster to GeoTIFF
            out_name = f"{Path(tif_name).stem}_clipped.tif"
            out_path = output_folder / out_name
            clipped.rio.to_raster(out_path)
            print(f"\nSaved: {out_path}\n")